# Global Settings

In [ ]:
DefaultSystemPrompt= """

You are Responding to a dating application profile,
Below is the current screen contents,
only say you love things we actually have in common.
be polite and concise, only comment on one or two things on their profile.
leave the message open ended for them to respond, we have already matched, Its okay to be flirty

""".strip()

# Functions

In [ ]:
import os
import subprocess
import time
import urllib.parse

In [ ]:
def adb(command):
    if type(command) == list:
        command = ' '.join(command)
    adb_command = f"adb {command}"
    return subprocess.run(adb_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True, text=True)

def adb_type(text: str):
    lines = text.split('\n')
    for i, line in enumerate(lines):
        # Send the line, replacing spaces with '%s'
        adb(f"shell input text '{line.replace(' ', '%s')}'")
        
        # Press ENTER key after each line, except for the last one
        if i < len(lines) - 1:
            adb("shell input keyevent 66")  # KeyCode 66 is the ENTER key
def TypeTextOnPhone(text):
    for ctext in list(str(text)):
        adb(["shell", "input", "text", ctext.replace(" ", "%s")])
        time.sleep(0.1)
    adb(["shell", "input", "keyevent", "66"])
    
def ocr_screen():
    # Take a screenshot
    r = adb("shell screencap /sdcard/screenshot.png")
    if r.returncode ==0:
        # Transfer the screenshot to your computer
        adb("pull /sdcard/screenshot.png")
        ocr_result_text='ERROR FETCHING'
        # Run OCR on the screenshot
        ocr_command = "tesseract screenshot.png output"
        ocr_result = subprocess.run(ocr_command, shell=True)
        if ocr_result.returncode == 0:
            with open("output.txt", "r") as output_file:
                return output_file.read().replace("|","I")
                print(my_profile_text)
        else:
            print("Error running OCR on the screenshot.")
    else:
        print("Screen protected")
        return None

In [ ]:
import openai
import keyring
openai.api_key =keyring.get_password("system", "openai_key")
def AskOpenAi(MyProfileText=None,TheirProfileText=None,AdditionalPrompt=None, BaseSystemPrompt=None):
    # Define the conversation messages
    
    messages=[]
    if MyProfileText is not None:
        messages.append({"role": "system", "content": f"""Role play like you are \n ```{MyProfileText}```"""})
        
    if TheirProfileText is not None:
        messages.append({"role": "system", "content": f"""The Profile you are responding to\n ```{TheirProfileText}```"""})
        
    if BaseSystemPrompt is None:
        messages.append({"role": "system", "content": DefaultSystemPrompt})
    else:
        messages.append({"role": "system", "content": BaseSystemPrompt})
          
    if AdditionalPrompt is not None:
        messages.append({"role": "user", "content": AdditionalPrompt})
        
    completion = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=messages
    )
    return completion.choices[0].message['content'].strip("'").strip()





# My Profile

In [ ]:
my_profile_text = ocr_screen()

 # Profile Commenter

In [ ]:
their_profile_text = ocr_screen()

In [ ]:

upgrades = AskOpenAi(
    MyProfileText=my_profile_text,
    BaseSystemPrompt="""
    
    Pretend you are a Matchmaker,
    you will provide a concise analysis of the profile and,
    tell me things that should be added that are typical of a dating profile
    
    """)
print(upgrades)

In [ ]:
red_flags = AskOpenAi(
    MyProfileText=my_profile_text,
    BaseSystemPrompt="""
    
    Pretend you are a Therapist and a Matchmaker,    
    bullet point any red flags you find in this profile
    
    """)
print(red_flags)

In [ ]:
red_flags = AskOpenAi(
    TheirProfileText=their_profile_text,
    BaseSystemPrompt="""
    
    Pretend you are a Therapist and a Matchmaker,    
    bullet point any red flags you find in this profile
    
    """)
print(red_flags)

In [ ]:
compatiblity = AskOpenAi(
    MyProfileText=my_profile_text,
    TheirProfileText=their_profile_text,
    BaseSystemPrompt=""""
    
    Pretend you are a Matchmaker,
    you will provide  a concise analysis the provided profiles 
    and give advice on how to proceed.
    
    """
)
print(compatiblity)

In [ ]:
introduction = askOpenAi(
    MyProfileText=my_profile_text,
    TheirProfileText=their_profile_text
)

print(introduction)

In [ ]:
things_to_talk_about = askOpenAiWhatToSay(
    MyProfileText=my_profile_text,
    TheirProfileText=their_profile_text,
    AdditionalPrompt="5 topics we could text about"
)

print(things_to_talk_about)

In [ ]:
first_dates = askOpenAiWhatToSay(
    MyProfileText=my_profile_text,
    TheirProfileText=their_profile_text,
    AdditionalPrompt="list of 5 first dates based on our profiles"
)

print(first_dates)

In [ ]:
#adb_type2(introduction)

In [ ]:
other = ocr_screen()

In [ ]:

    adb("shell screencap /sdcard/screenshot.png")